# Matching movies for a balanced dataset

IMPORTANT REMARK: this notebook can't be run, it is a draft for matching after we finish the adjustements and the cleaning of cofounders analysis

## Calcul of propensity scores

In [3]:
import statsmodels.api as sm
import statsmodels.formula.api as smf


### First step: transform success into binary data

Why? Currently, matching methods rely on the match between one "control" point and one "treated" point. Research is aiming to develop propensity score calculation methods on non-binary treatment (but we are far away from being data analysis researchers...). We thus need to binarize our success indicator. This will unfortunately limit our conclusions and simplify a lot (too much?) our study.

In [ ]:
#Perform logistic regression with cofounders found previously

mod == smf.logit(formula='IsSuccessful ~  C(HasFamousActor) + C(IsFamousDirector) +\
                        C(IsFamousStudio) ', data=success_data) #to complete with more cofounders?

res = mod.fit()
print(res.summary())


In [ ]:
#Add propensity score to the dataframe
success_data['Propensity_score']=res.predict()

In [ ]:
#get similarity
def similarity(propensity_x,propensity_y):
    return 1-np.abs(propensity_x-propensity_y)

In [ ]:
#Perform matching

import networkx as nx
G=nx.Graph()
movies_successful=success_data[success_data.IsSuccessful==1]
movies_non_successful=success_data[success_data.IsSuccessful==0]
for non_successful_id, non_successful_row in movies_non_successful.iterrows():
    
    for successful_id, successful_row in movies_successful.iterrows():
        
        s=similarity(non_successful_row['Propensity_score'],successful_row['Propensity_score'])
    
        G.add_weighted_edges_from([(non_successful_id,successful_id,s )])
        
matching = nx.max_weight_matching(G)

In [ ]:
matched = [i[0] for i in list(matching)] + [i[1] for i in list(matching)]
balanced_success_data = df.iloc[matched]
balanced_success_data.head()